In [1]:
import pandas as pd
import os, glob
import pickle
import numpy as np
from obspy.taup import TauPyModel
from obspy.geodetics import kilometers2degrees, gps2dist_azimuth
model = TauPyModel(model="iasp91")

In [2]:
PKLpath = '/Volumes/home/Research/STEP/01_Split'
Path = '/Volumes/home/Research/STEP/02_Station_result_csv'
SAVEpath = '.'
# RElook = []
RElook= ['BYUR','GUDG','TBLG','KIV','GNI','ONI','AKH']
# RElook = ['ALAV','ARZA',]
YearRange = '2010-2020'
ver=1

In [7]:
def Calc_rho(RCdt, SCdt):
    rho = RCdt/SCdt 
    return rho
def Calc_Phi(RCPhi, SCPhi):
    Phi = max (abs(RCPhi-SCPhi), abs(SCPhi-RCPhi))
    if Phi > 90: Phi = 180 - Phi
    return Phi
def classify(df,stlat,stlon):
    phiSC = []; dtSC  = []
    NULL  = []; Phase = []
    p_lat = []; p_lon = []
    e_lat = []; e_lon = []
    for i in range(len(df)):
        evlat = df['Ev_lat'].values[i]
        evlon = df['Ev_lon'].values[i]
        depth = df['Depth'].values[i]
        phase = df['Phase'].values[i]
        RCPhi = df['RCPhi'].values[i]
        RCDPhi  = df['RCPhi_std'].values[i]
        RCdt  = df['RCdt'].values[i]
        RCDdt  = df['RCdt_std'].values[i]
        SCPhi = df['SCPhi'].values[i]
        SCDPhi  = df['SCPhi_std'].values[i]
        SCdt  = df['SCdt'].values[i]
        SCDdt  = df['SCdt_std'].values[i]

        rho = Calc_rho(RCdt, SCdt)
        phi = Calc_Phi(RCPhi, SCPhi)
        if 25 < phi < 68 or df["CpH"].values[i] > 0.76 :
            NULL.append('True')
            cmd = """
            taup pierce -mod iasp91 -h %(depth)f -sta %(stlat)f %(stlon)f  -evt %(evlat)f %(evlon)f -ph %(phase)s --pierce 2700 -nodiscon >temp.txt
            """ % locals()
            os.system(cmd)
            cmd = """
            cat temp.txt | awk 'NR==3 {print$4, $5}'
            """ % locals()
            p=os.popen(cmd)
            text =str(p.read())
            p.close()
            p_lat.append(text.rsplit(' ')[0])
            p_lon.append(text.rsplit(' ')[1][:-1])  
            os.system('rm -f temp.txt')
            e_lat.append(evlat); e_lon.append(evlon)
            Phase.append(phase)
            phiSC.append(SCPhi); dtSC.append(SCdt)
        elif 0.8 < rho < 1.1 and phi < 8:## non-null good condition 
            NULL.append('False')
            cmd = """
            taup pierce -mod iasp91 -h %(depth)f -sta %(stlat)f %(stlon)f  -evt %(evlat)f %(evlon)f -ph %(phase)s --pierce 2700 -nodiscon >temp.txt
            """ % locals()
            os.system(cmd)
            cmd = """
            cat temp.txt | awk 'NR==3 {print$4, $5}'
            """ % locals()
            p=os.popen(cmd)
            text =str(p.read())
            p.close()
            p_lat.append(text.rsplit(' ')[0])
            p_lon.append(text.rsplit(' ')[1][:-1])  
            os.system('rm -f temp.txt')
            e_lat.append(evlat); e_lon.append(evlon)
            Phase.append(phase)
            phiSC.append(SCPhi); dtSC.append(SCdt)
        elif 0.7 <= rho < 1.2 and phi <= 25: ## non-null FAIR condition 
            NULL.append('False')
            cmd = """
            taup pierce -mod iasp91 -h %(depth)f -sta %(stlat)f %(stlon)f  -evt %(evlat)f %(evlon)f -ph %(phase)s --pierce 2700 -nodiscon >temp.txt
            """ % locals()
            os.system(cmd)
            cmd = """
            cat temp.txt | awk 'NR==3 {print$4, $5}'
            """ % locals()
            p=os.popen(cmd)
            text =str(p.read())
            p.close()
            p_lat.append(text.rsplit(' ')[0])
            p_lon.append(text.rsplit(' ')[1][:-1])  
            os.system('rm -f temp.txt')
            e_lat.append(evlat); e_lon.append(evlon)
            Phase.append(phase)
            phiSC.append(SCPhi); dtSC.append(SCdt)            
        else: 
            pass
    result_df = {'phiSC': phiSC, 
                 'dtSC': dtSC,
                 'p_lat': p_lat,
                 'p_lon': p_lon,
                 'null': NULL,
                 'phase':Phase,
                 'e_lat':e_lat,
                 'e_lon':e_lon}
    return result_df

In [9]:
for STApath in glob.glob(f'{PKLpath}/AB*.pkl'):
    
    STApkl = pickle.load(open(STApath, "rb"))
    values = [value for keys, value in STApkl.items() ][0]

    NET = values['network']; STA = values['station']
    if STA not  in RElook:
        stlat = values['latitude']; stlon = values['longitude']
        SKSdf = pd.DataFrame()
        for csvpath in glob.glob(f'{Path}/*SKS/{NET}*{STA}*.csv'):
            df = pd.read_csv(csvpath)
            df = df[df['SNRQ']>=5][df['Pick']==True][df['Phase']=='SKS']
            SKSdf = SKSdf.append(df)     
        SKSresult_df= classify(SKSdf,stlat,stlon)

        SKKSdf = pd.DataFrame()
        for csvpath in glob.glob(f'{Path}/*SKKS/{NET}*{STA}*.csv'):
            df = pd.read_csv(csvpath)
            df = df[df['SNRQ']>=5][df['Pick']==True][df['Phase']=='SKKS']
            SKKSdf = SKKSdf.append(df) 
        SKKSresult_df= classify(SKKSdf,stlat,stlon)
        
        SKSresult_df = pd.DataFrame(SKSresult_df)
        SKKSresult_df = pd.DataFrame(SKKSresult_df)
        res = pd.concat([SKSresult_df,SKKSresult_df],axis=0)
        res.to_csv(f'{NET}.{STA}_PiercePoint.csv',index=False)
        

/var/folders/sp/pnngww1n0q31w2plgbk53rc00000gn/T/ipykernel_86544/431839437.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['SNRQ']>=5][df['Pick']==True][df['Phase']=='SKS']


In [6]:
res

,phiSC,dtSC,p_lat,p_lon,null,phase,e_lat,e_lon
0,15.79,2.7,41.37,34.00,1.0,SKS,-15.3083,-65.1634
